In [ ]:
import os

import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from rasterio.features import geometry_mask, rasterize
from rasterio.transform import from_bounds
from shapely.geometry import mapping

In [ ]:
labels_dir = "../data/labels"
masks_dir = "../data/masks"
metadata_file = "../data/metadata.csv"
df_meta = pd.read_csv(metadata_file)
df_meta.info()

In [ ]:
# add new filename map column to find metadata for each image
df_meta["filename_map"] = df_meta["filename"].str.replace("_ortho.tif", "")
df_meta.head()

In [ ]:
# assert there is metadata for all gpkg files in given directory
for filename in os.listdir(labels_dir):
    assert filename.endswith(".gpkg")
    assert (
        filename.replace("_ortho_polygons.gpkg", "") in df_meta["filename_map"].values
    )

In [ ]:
# iterate over all gpkg files in given directory
for filename in os.listdir(labels_dir):
    if filename.endswith(".gpkg"):
        filepath = os.path.join(labels_dir, filename)

        # Get metadata for current gpkg file
        filename_map = filename.replace("_ortho_polygons.gpkg", "")
        file_meta = df_meta.loc[df_meta["filename_map"] == filename_map].to_dict(
            "records"
        )[0]
        out_image = np.zeros((file_meta["height"], file_meta["width"]), dtype=np.uint8)

        # crop the image based on the extend of the aoi layer
        gdf_aoi = gpd.read_file(filepath, layer="aoi")

        # Read in gpkg file and determine if standing deadwood is present
        layers = fiona.listlayers(filepath)
        if "standing_deadwood" in layers:
            gdf_label = gpd.read_file(filepath, layer="standing_deadwood")

            # Create transform from metadata
            transform = from_bounds(
                west=file_meta["left"],
                east=file_meta["right"],
                north=file_meta["top"],
                south=file_meta["bottom"],
                width=file_meta["width"],
                height=file_meta["height"],
            )

            # Rasterize polygons
            for _, row in gdf_label.iterrows():
                geom = mapping(row["geometry"])
                mask = geometry_mask(
                    [geom], transform=transform, invert=True, out_shape=out_image.shape
                )
                out_image[mask] = 1

            # Save image
            with rasterio.open(
                os.path.join(masks_dir, filename_map + "ortho_mask.tif"),
                "w",
                driver="GTiff",
                height=out_image.shape[0],
                width=out_image.shape[1],
                count=1,
                dtype="uint8",
                crs="EPSG:4326",
                transform=transform,
            ) as dst:
                dst.write(out_image, 1)